In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from sklearn.metrics import accuracy_score, classification_report
from textpreprocessor import TextPreprocessor

# Initialize the Text Pre Processor class
processor = TextPreprocessor()

# Load data
df_train, df_test = processor.load_data()

# Preprocess data
df_train = processor.preprocess(df_train)
df_test = processor.preprocess(df_test)

# Split data
X_train, y_train = processor.split_data(df_train)
X_test, y_test = processor.split_data(df_test)

(3600000, 3)
(10000, 3)
(400000, 3)
(10000, 3)


In [3]:
X_train_pad, X_test_pad = processor.tokenization_and_padding(X_train, X_test)

In [4]:
# RNN Model Building
model = Sequential()
model.add(Embedding(input_dim=processor.max_features, output_dim=128, input_length=processor.max_length))
model.add(SimpleRNN(64))  # Using a Simple RNN layer
model.add(Dense(1, activation='sigmoid'))  # Binary classification

/home/woong/myenv/lib/python3.9/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1726777706.026156   65795 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1726777706.051914   65795 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1726777706.051981   65795 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1726777706.056233   65795 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1726777706.056290   65795 

In [5]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [6]:
# Train the model
model.fit(X_train_pad, y_train, epochs=5, batch_size=64, validation_data=(X_test_pad, y_test))

Epoch 1/5


I0000 00:00:1726777707.055917   66190 service.cc:146] XLA service 0x7f34b8005b50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726777707.055950   66190 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-09-19 22:28:27.074562: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-19 22:28:27.159760: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


  7/157 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.4961 - loss: 0.6928

I0000 00:00:1726777707.468568   66190 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


157/157 ━━━━━━━━━━━━━━━━━━━━ 7s 36ms/step - accuracy: 0.5795 - loss: 0.6600 - val_accuracy: 0.7624 - val_loss: 0.4954
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.8552 - loss: 0.3472 - val_accuracy: 0.7835 - val_loss: 0.4840
Epoch 3/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.9677 - loss: 0.1193 - val_accuracy: 0.7863 - val_loss: 0.5769
Epoch 4/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - accuracy: 0.9945 - loss: 0.0305 - val_accuracy: 0.7660 - val_loss: 0.7236
Epoch 5/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.9994 - loss: 0.0092 - val_accuracy: 0.7704 - val_loss: 0.7866


In [7]:
# Evaluate the model
y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.5).astype("int32")

313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step


In [8]:
# Model evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred))

Model Accuracy: 77.04%
Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.75      0.77      4972
           1       0.76      0.79      0.77      5028

    accuracy                           0.77     10000
   macro avg       0.77      0.77      0.77     10000
weighted avg       0.77      0.77      0.77     10000

